In [ ]:
DESIRED_PASSWORD = 

In [1]:
import requests
from guerrillamail import GuerrillaMailSession
from bs4 import BeautifulSoup
import time
import random
import re
import pandas as pd

In [2]:
mail_session = GuerrillaMailSession()
email = mail_session.get_session_state()['email_address']

In [3]:
vote_url = 'https://usa.tycoononline.com/vote.php'
homepage = 'https://usa.tycoononline.com/frame_index.php'

In [4]:
def get_savings():
    r = s.get(homepage)
    raw_innerHTML =  BeautifulSoup(r.text).find('table', {'id': 'header'}).find_all('td', {'class': 'text8 info'})[0].text
    int_innerHTML = re.findall(r'\d+', raw_innerHTML)
    full_str = ''
    for i in int_innerHTML:
        full_str += i
    return int(full_str)

In [5]:
def try_vote():
    r = s.get(vote_url)
    return r

In [6]:
df = pd.read_csv('accounts.csv')

In [7]:
accNum = len(df) + 16
print(len(df))

10


In [8]:
print(email)

jpvbquas@guerrillamailblock.com


## Account creation

In [9]:
url = 'https://usa.tycoononline.com/frame_index.php?page=regMedlem1'
s = requests.Session()
r = s.get(url)
sid = r.cookies['PHPSESSID']

In [10]:
#Submit email address
params = {
    'sidvariabel': sid,
    'epost': email
}
r = s.post('https://usa.tycoononline.com/regMedlem1Kontroll.php', data = params)

In [11]:
#Retrieve registration email
registration_email = None
intervals = 0
print("Waiting for registration email:")
while not registration_email:
    emails = mail_session.get_email_list()
    for mail in emails:
        if 'tycoononline.com' in mail.sender:
            registration_email = mail
    print(intervals)
    time.sleep(5)
    intervals += 1
print('Waited {} intervals.\n'.format(intervals))

Waiting for registration email:
0
1
2
3
Waited 4 intervals.



In [13]:
registration_email_body = mail_session.get_email(registration_email.guid).body
registration_link = BeautifulSoup(registration_email_body).find('a')['href']
r = s.get(registration_link)

In [14]:
date = str(random.randint(1,28))
month = str(random.randint(1,12))
year = str(random.randint(1950,2000))
password = DESIRED_PASSWORD
firstName = 'mitch' + str(random.randint(1,1000))
lastName = 'laskey'
country = '18'
secName = 'Micheal'

In [15]:
tejaFirst = 'Tycoon'
tejaLast = 'Multi ' + str (accNum)
tejaGender = str(random.randint(0,1))
tejaOccupation = str(random.randint(1,3))
tejaCountry = str(random.randint(1,150))

tejaCompanyName = chr(65+accNum) * 4

In [15]:
#Register
registration_params = {
    'sidvariabel': sid,
    'regform': 'true',
    'dob': date,
    'mob': month,
    'yob': year,
    'password': password,
    'password2': password,
    'firstname': firstName,
    'lastname': lastName,
    'gender': '1', # 1 or 0
    'occupation': '3',#1,2,3?
    'showage': '2',
    'countryId': country, #randomize?
    'touch': '0',
    'secretary': secName,
    'agreetoterms': 'yes',
    'submitBtn': 'Next+>>'
}

r = s.post(registration_link, data = registration_params)

## Company creation

In [18]:
company_params = {
    'companyName': tejaCompanyName,
    'specialitet': '7' #1 is wood, 5 is stone, 7 is wheat
}
make_company_page = 'https://usa.tycoononline.com/regCompany1Kontroll.php'
r = s.post(make_company_page, data = company_params)

In [19]:
city = str(random.randint(1,9))

In [20]:
city_params = {
    'cityId': city
}
choose_city_page = 'https://usa.tycoononline.com/regCompany2Kontroll.php'
r = s.post(choose_city_page, data = city_params)

## Get money

 - Take out loan
 - Sell shares

In [21]:
#Take out loan
loan_params = {
    'summa': '5000',
    'veckor': '1'
}

loan_page = 'https://usa.tycoononline.com/lanNyttInsert.php'
r = s.post(loan_page, data = loan_params)

In [22]:
own_shares_url = 'https://usa.tycoononline.com/frame_index.php?page=aktieMina'

r = s.get(own_shares_url)

In [23]:
#Find share ids
share_ids = []
share_ids_page = BeautifulSoup(r.text)
share_inputs = share_ids_page.find_all('input', {'name':'akties[]'})
for share in share_inputs:
    share_ids.append(share['value'])

In [24]:
#Sell shares
sell_shares = {
    "all":"true", #try with only this?
    "akties[]": share_ids,
    "sellToBank":"Instantly+sell+to+bank+for+a+reduced+price"
}

sell_shares_url = 'https://usa.tycoononline.com/aktieSalj.php'
r = s.post(sell_shares_url, sell_shares)

## Build factory

In [25]:
def find_first_buildable_coords(map_buildable, building_size):
    for row in range(len(map_buildable)):
        for tile in range(len(map_buildable[row])):
            if map_buildable[row][tile]:
                found_occupied = False
                for subrow in range(row, row + building_size):
                    if subrow >= len(map_buildable):
                        break
                    for subcol in range(tile, tile + building_size):
                        if subcol >= len(map_buildable):
                            break
                        if not map_buildable[subrow][subcol]:
                            found_occupied = True
                if not found_occupied:
                    return (tile, row)
    return None

In [26]:
def is_buildable(tile):
    attributes = tile.find('img')['onmouseover'].split(',')
    company = attributes[3]
    land_type = attributes[-1]
    return company == "''" and 'Countryside' in land_type

In [27]:
def build_and_place_factory(factory_id, building_size, typ = 2):
    """
    factory_id: Forest is 1, Sawmill is 4, Wheat field is 7, Flourmill is 8
    building_size: Size of the building. If irregular, this should be the longest side.
    typ: 2 is Countryside. Currently only works for countryside buildings
    
    Returns byggandId, which is building id.
    """
    factory_id = str(factory_id)
    typ = str(typ)
    build_factory_params = {
        'fabriktypId': factory_id
    }
    build_factory_page = 'https://usa.tycoononline.com/fabrikBygg.php?fabriktypId={}'.format(factory_id)
    r = s.post(build_factory_page, data = build_factory_params)
    
    
    build_factory_response = BeautifulSoup(r.text)
    args = str(build_factory_response.find(id='placeBuilding2').parent).split('=')
    kartabyggnadId = args[3].split('&')[0]
    byggandId = args[4].split('&')[0]
    

    place_factory_1_params = {
        "page": "kartaSegment",
        "kartabyggnadId": kartabyggnadId,
        "byggnadId": byggandId,
        "typ": typ}

    place_factory_url = 'https://usa.tycoononline.com/frame_index.php'
    r = s.post(place_factory_url, data = place_factory_1_params)
    
    
    active_tile_img = '../../bilder/z/g/l/10_1.gif' #Countryside only
    less_active_tile_img = '../../bilder/z/g/l/10_2.gif'
    least_active_tile_img = '../../bilder/z/g/l/10_3.gif'
    place_factory_1_response = BeautifulSoup(r.text)
    active_tiles = place_factory_1_response.findAll("img", {"src" : active_tile_img})
    if not active_tiles:
        active_tiles = place_factory_1_response.findAll("img", {"src" : less_active_tile_img})
        if not active_tiles:
            active_tiles = place_factory_1_response.findAll("img", {"src" : least_active_tile_img})
    tile_args = active_tiles[0].parent['onclick'].split(',')
    x = re.findall(r'\d+', tile_args[0])[0]
    y = re.findall(r'\d+', tile_args[1])[0]
    
    
    place_factory_2_params = {
        "page":"kartaPlaceraByggnad1",
        "X": x,
        "Y": y,
        "kartabyggnadId": kartabyggnadId,
        "byggnadId": byggandId}
    r = s.post(place_factory_url, data = place_factory_2_params)
    place_factory_2_response = BeautifulSoup(r.text)
    kartagrovId = place_factory_2_response.find('input', {'name': 'kartagrovId'})['value']
    long_js = place_factory_2_response.find('td', {'id': 'main-column'}).find_all('script')[0]
    p = re.compile('kartaIdStart = (\d*);')
    kartaIdStart = p.search(str(long_js))[0]
    kartaIdStart = re.findall(r'\d+', kartaIdStart)[0]
    
    
    check_map_url = 'https://usa.tycoononline.com/frame_index.php?page=kartaTitta&X={}&Y={}&cityId={}'.format(x,y,city)
    r = s.get(check_map_url)
    

    check_map = BeautifulSoup(r.text)
    full_map = check_map.find('td', {'id': 'main-column'}).find_all('table')[5].find('tr').find_all('tr') #Elements are rows
    found_spot = False
    #full_map[curr_y].find_all('td')[curr_x].find('img')['onmouseover'].split(',')
    
    full_map_buildable = []
    for row in full_map:
        tiles = row.find_all('td')
        row_buildable = list(map(is_buildable, tiles))
        full_map_buildable.append(row_buildable)
        
    inner_xy_pair = find_first_buildable_coords(full_map_buildable, building_size)
    inner_x = inner_xy_pair[0]
    inner_y = inner_xy_pair[1]
    kartaId = str(int(kartaIdStart) + inner_x + (inner_y * 20))
    
    choose_tile_params = {
        "rutnat": "0", #Grid - Show grid lines, can always be 0
        "kartagrovId": kartagrovId, #Rough map - In js within tile page - Transformed x,y coords?
        "kartaId": kartaId, #Map - Found in js of bigger tile page
        "kartabyggnadId": kartabyggnadId, #Map building - Found in factory page
        "byggnadId": byggandId, #Building - Building id (fabrikId) Found in js of bigger map
        "X": x,
        "Y": y
    }
    choose_tile_url = 'https://usa.tycoononline.com/kartaPlaceraByggnad2.php'
    r = s.post(choose_tile_url, choose_tile_params)
    
    return byggandId

In [28]:
wheat_field_01_id = build_and_place_factory(7, 3)

AttributeError: 'NoneType' object has no attribute 'parent'

In [ ]:
wheat_field_02_id = build_and_place_factory(7, 3)

## Hire and assign staff

In [ ]:
def hire_and_place_staff(profession_id, building_id):
    """
    profession_id: 15 is sawmill worker, 10 is woodcutter, 2 is farmer, 8 is miller
    building_id: same as byggnadId
    """
    post_job_ad  = {
        "page":"hirestaff",
        "step":"1",
        "profession": str(profession_id)
    }
    post_job_ad_url = 'https://usa.tycoononline.com/frame_index.php?page=hirestaff'
    r = s.post(post_job_ad_url, post_job_ad)
    
    
    staff_labels = BeautifulSoup(r.text).find_all('label')
    staff_ids = []
    for i in range(3):
        rawAttribute = staff_labels[i]['for']
        numAttribute = str(re.findall(r'\d+', rawAttribute)[0])
        staff_ids.append(numAttribute)
        
        
    hire_staff = {
        "page":"hirestaff",
        "step":"2",
        "hire[]": staff_ids
    }
    hire_staff_url = 'https://usa.tycoononline.com/frame_index.php?page=hirestaff'
    r = s.post(hire_staff_url, hire_staff)
    
    
    for staff_id in staff_ids:
        place_staff = {
            "personId": staff_id,
            "byggnadId": '1' + str(building_id)
        }
        place_staff_url = 'https://usa.tycoononline.com/personByggnad.php'
        r = s.post(place_staff_url, place_staff)

In [ ]:
#Hire and place sawmill workers
hire_and_place_staff(2, wheat_field_01_id)
hire_and_place_staff(2, wheat_field_02_id)

In [ ]:
current_savings = get_savings()

In [ ]:
row = [email, current_savings, 2, 500]
df.loc[len(df)] = row
df.to_csv('accounts.csv', index=False)